In [1]:
#import packages
#os to fast track terminal use
import os
os.system("pip install pandas spacy")
#spacy for nlp
import spacy
os.system("python -m spacy download en_core_web_md")
nlp = spacy.load('en_core_web_md')
#pandas for dataframes
import pandas as pd
#numpy for hjelp
import numpy as np
#for cleaning data
import re

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 700.0 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 1.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 53.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 38.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 47.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.9/156.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.8/394.8 kB 7.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 27.8 MB/s 


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 23.7 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
/tmp/ipykernel_540/454231508.py:10: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#get file
#filepath set for sparately added Ucloud data directory
fp = "/work/cds-lang-data/USEcorpus/USEcorpus/"
#os independent filepath

#filepath for hand-in?


In [100]:
# function for 1 file from 1 folder

def count_ofof(fp, extract_this = ["NOUN","VERB","ADJ","ADV"],
 ent_list = ["PERSON","GPE","ORG"], clean_metadata = '<.*?>'):
    #load it with encoding 
    with open(fp, encoding="latin-1") as f:
        text = f.read()

    #clean metadata 
    text_c = re.sub(clean_metadata, '', text)

    #convert it with spacy
    text_nlp = nlp(text_c)

    #extract values, I have to use "i" to  index items, long words confuse me
    #empty list for extracts
    extract = []
    for i in text_nlp:
        if i.pos_ in extract_this:
            extract.append(i.pos_)
        else:
            pass 
    
    #get named entities out
    ent_extract = []
    for i in text_nlp.ents:
        if i.label_ in ent_list:
            ent_extract.append(i.label_)
        else:
            pass
    
    #add lists if there were none
    missing_pos, missing_pos_zeros = addmissing(extract,extract_this)
    missing_ents, missing_ents_zeros = addmissing(ent_extract, ent_list)

    #regroup
    extract = extract + ent_extract
    missing_pos = missing_pos + missing_ents
    missing_pos_zeros = missing_pos_zeros + missing_ents_zeros

    #returning counts of unique occurences
    if len(missing_pos) == 0:
        #returns count extracts
        count_extracts = np.unique(extract, return_counts= True )
    else:
        count_extracts = np.unique(extract, return_counts= True )  

        #appends missing pos's
        fos = np.append(count_extracts[0],missing_pos)
        fos_zeros = np.append(count_extracts[1],missing_pos_zeros)
        fos_full = (fos,fos_zeros)

        count_extracts = fos_full

    return(count_extracts)

In [3]:
#turn it to dictionary, because I want to practice working with those
def dict_it(count_extracts):
    dict_extracts = {}
    dict_list =[]
    for i in range(len(count_extracts[1])):
        if i == 0:
            dict_extracts["pos"] = count_extracts[0][0]
            dict_extracts["count"] = count_extracts[1][0]
            dict_list.append(dict_extracts)
        else:
            upd_ext = {"pos":count_extracts[0][i],"count":count_extracts[1][i]}
            dict_list.append(upd_ext)
    return(dict_list)


In [4]:
#in case the POS is not in the text, it would return 
#an array with one less item and that could scramble things
#so in case the asked POS is not there this part adds it as 0

def addmissing(extract,list_of_stuff):
    missing_stuff = []
    for i in list_of_stuff:
        if i not in extract:
            missing_stuff.append(i)
        else:
            pass
    #making an equal length list of 0s
    missing_stuff_zeros = []
    for i in missing_stuff:
        missing_stuff_zeros.append(0)
    
    return(missing_stuff,missing_stuff_zeros)

In [5]:
# multiple files from 1 folder
def count_mfof(fp):
    fp_s = sorted(os.listdir(fp))
    dict_list_list = []
    #get count out of the file
    for i in fp_s: 
        extract_array = count_ofof(fp + i)
        extract_dict = dict_it(extract_array)
        #add filename to each dict in list to prevent confusion on where the data came from
        for j in extract_dict:
            j.update({"file":i})
    
        dict_list_list.append(extract_dict)
    return(dict_list_list)    



In [6]:
#extract multiple files from multiple

def count_mfmf(fp):
    #sort filenames
    fp_s = sorted(os.listdir(fp))
    #empty list for storing results
    folder_list = []
    #get count out of the folders
    for i in fp_s: 
        #use the many file one folder function to extract counts
        extract_array = count_mfof(fp + i + "/")
        
        #to avoid confusion folder name will also be inserted into the dictionaries
        #for all files 
        for f in range(0,len(extract_array)):
            #into each dictionary of each folder of each file insert folder name
            for j in extract_array[f]:
                j.update({"folder" : i })

        #add results to list
        folder_list.append(extract_array)
        
    return(folder_list)    


In [ ]:
#for formatting to wanted table structure
def table_format(one_file):
    df = pd.DataFrame(one_file)
    #pivot to wanted form
    df = df.pivot(index="file", columns = "pos", values = "count")
    #reorder to wanted form
    df = df[["NOUN","VERB","ADJ","ADV","PERSON","GPE","ORG"]]
    #make calculations
    df[["NOUN","VERB","ADJ","ADV"]] = df[["NOUN","VERB","ADJ","ADV"]].div(10000)
    #rename to wanted form
    df = df.rename(columns={"NOUN" : "RelFreq NOUN",
    "VERB": "RelFreq VERB",
    "ADJ" : "RelFreq ADJ",
    "ADV" : "RelFreq ADV",
    "PERSON" : "No. Unique PER",
    "GPE" : "No. Unique LOC",
    "ORG" : "No. Unique ORG",})
    #make it nice finishing touches, remove pos index, add filename
    df = df.reset_index()
    df = df.rename(columns={"file" : "Filename"})
    df = df.rename_axis(None, axis=1)

    return(df)


In [65]:
fp = "/work/cds-lang-data/USEcorpus/USEcorpus/"
test_2 = count_mfmf(fp)

In [102]:
##### TO DO
## how to add pere loc org ------ Done
## what happens when no noun verb etc in text?---- DONE
## turn output to freq ------ DONE
## make it into a nice table(s) ------ Done
## clean metadata ----DONE
##call functions from separate file
## save it


In [103]:
fp = "/work/cds-lang-data/USEcorpus/USEcorpus/c1/"

test = count_mfof(fp)
df = table_format(test[0])

In [104]:
#Formatting values to table for all files in a table
cter = 0
for i in test:
    if cter == 0:
        df_perm = table_format(i)
    else:
        df_temp = table_format(i)
        df_perm = pd.concat([df_perm,df_temp], ignore_index = True)
    
    cter = cter+1

df_perm


,Filename,RelFreq NOUN,RelFreq VERB,RelFreq ADJ,RelFreq ADV,No. Unique PER,No. Unique LOC,No. Unique ORG
0,0140.c1.txt,0.0386,0.0229,0.0116,0.0099,78,35,8
1,0165.c1.txt,0.0429,0.0201,0.0143,0.0070,52,17,6
2,0200.c1.txt,0.0234,0.0203,0.0129,0.0101,79,4,10
3,0219.c1.txt,0.0208,0.0147,0.0085,0.0073,72,16,6
4,0238.c1.txt,0.0140,0.0149,0.0051,0.0037,61,3,3
5,0501.c1.txt,0.0179,0.0149,0.0067,0.0062,54,7,6
6,0502.c1.txt,0.0207,0.0191,0.0068,0.0064,40,22,26
